In [1]:
'Test_Task3_Baby presence detection when covered in blanket or sunscreen'

'Test_Task3_Baby presence detection when covered in blanket or sunscreen'

In [2]:
import gc
gc.collect()

3

In [3]:
%reset -f

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing

# Test measurements

In [5]:
import os
import numpy as np
import pandas as pd
# Replace 'your_array_file.npy' with the actual file path of your saved NumPy array
# file_path = 'BabyCovered_with_Blanket_or_Sunscreen_npy_array_Lowpassfiltered_label.npy'

file_path = os.path.join("..", "..", "Data", "Processed", "BabyCovered_with_Blanket_or_Sunscreen_npy_array_Lowpassfiltered_label.npy")


# Load the NumPy array from the file
loaded_array = np.load(file_path, mmap_mode='r')

# Now 'loaded_array' contains the NumPy array data that was saved in the file
dataframe_testdata = pd.DataFrame(loaded_array,columns=['Frequency','FFT Magnitude','Phase','Infant_Presence'])
#dataframe_testdata = dataframe_testdata.iloc[:2000]
#dataframe_testdata = dataframe_testdata.sample(n=2000, random_state=42)
dataframe_testdata

,Frequency,FFT Magnitude,Phase,Infant_Presence
0,0.000000,48274.236998,0.000000,1.0
1,162.760417,24941.596497,-3.088069,1.0
2,325.520833,1716.246635,0.987059,1.0
3,488.281250,759.834659,-1.668255,1.0
4,651.041667,1026.975587,2.278972,1.0
...,...,...,...,...
11995,-813.802083,1362.883093,0.664906,1.0
11996,-651.041667,1026.975587,-2.278972,1.0
11997,-488.281250,759.834659,1.668255,1.0
11998,-325.520833,1716.246635,-0.987059,1.0


In [6]:
num_rows_test = len(dataframe_testdata)
num_rows_test

12000

In [7]:
fft_magnitudes_test = dataframe_testdata["FFT Magnitude"].values  # Convert to NumPy array
fft_magnitudes_test

array([48274.23699804, 24941.59649665,  1716.24663508, ...,
         759.83465866,  1716.24663508, 24941.59649665], shape=(12000,))

In [8]:
fft_frequencies_test = dataframe_testdata["Frequency"].values  # Convert to NumPy array
fft_frequencies_test

array([   0.        ,  162.76041667,  325.52083333, ..., -488.28125   ,
       -325.52083333, -162.76041667], shape=(12000,))

In [9]:
fft_phase_test = dataframe_testdata["Phase"].values  # Convert to NumPy array
fft_phase_test

array([ 0.        , -3.08806921,  0.98705878, ...,  1.66825465,
       -0.98705878,  3.08806921], shape=(12000,))

In [10]:
import numpy as np
from scipy.stats import entropy
from scipy.signal import find_peaks

def extract_features(fft_freqs, fft_mags, fft_phase):
    # Normalize magnitudes to prevent division errors
    norm_mags = fft_mags / np.sum(fft_mags) if np.sum(fft_mags) > 0 else fft_mags
    
    # Spectral Centroid (Weighted Mean of Frequencies)
    spectral_centroid = np.sum(fft_freqs * norm_mags) / np.sum(norm_mags)

    # Spectral Bandwidth (Spread around centroid)
    spectral_bandwidth = np.sqrt(np.sum(norm_mags * (fft_freqs - spectral_centroid) ** 2))

    # Spectral Flatness (Geometric Mean / Arithmetic Mean)
    spectral_flatness = np.exp(np.mean(np.log(fft_mags + 1e-10))) / np.mean(fft_mags + 1e-10)

    # Find peaks in the magnitude spectrum
    peaks, _ = find_peaks(fft_mags, height=0.1 * np.max(fft_mags))  # Adaptive threshold

    # Harmonic Ratio (Ratio of 2nd peak to 1st peak)
    if len(peaks) >= 2:
        harmonic_ratio = fft_mags[peaks[1]] / fft_mags[peaks[0]]
    else:
        harmonic_ratio = 0  # No second peak detected

    # Phase Features
    phase_variance = np.var(fft_phase)  # Variance of phase angles
    phase_mean = np.mean(fft_phase)  # Mean of phase angles
    phase_diff = np.mean(np.diff(fft_phase))  # Average phase difference between frequencies

    return {
        "mean_fft": np.mean(fft_mags),
        "std_fft": np.std(fft_mags),
        "max_fft": np.max(fft_mags),
        "min_fft": np.min(fft_mags),
        "median_fft": np.median(fft_mags),
        "sum_fft": np.sum(fft_mags),
        "spectral_entropy": entropy(norm_mags),  # Energy spread
        "spectral_centroid": spectral_centroid,
        "spectral_bandwidth": spectral_bandwidth,
        "spectral_flatness": spectral_flatness,
        "phase_variance": phase_variance,
        "phase_mean": phase_mean,
        "phase_diff": phase_diff,
    }

In [11]:
# Function to apply perturbation based on reference values
def add_perturbation(reference_features, num_rows, perturb_range=0.03):
    perturbed_data = []
    
    for _ in range(num_rows):
        perturbed_features = {
            key: value * (1 + np.random.uniform(-perturb_range, perturb_range)) 
            for key, value in reference_features.items()
        }
        perturbed_data.append(perturbed_features)
    
    return pd.DataFrame(perturbed_data)

In [12]:
# Extract features for fft_test
reference_features_test = extract_features(fft_frequencies_test, fft_magnitudes_test, fft_phase_test)
reference_features_test

{'mean_fft': np.float64(539.3153031280609),
 'std_fft': np.float64(624.5127144952429),
 'max_fft': np.float64(48274.23699803697),
 'min_fft': np.float64(4.254391050121097),
 'median_fft': np.float64(478.6752374645231),
 'sum_fft': np.float64(6471783.637536731),
 'spectral_entropy': np.float64(9.177761059479113),
 'spectral_centroid': np.float64(-59.48645822846447),
 'spectral_bandwidth': np.float64(567668.4758818537),
 'spectral_flatness': np.float64(0.8096637590819117),
 'phase_variance': np.float64(3.267537662509709),
 'phase_mean': np.float64(2.960594732333751e-17),
 'phase_diff': np.float64(0.0002573605476758946)}

In [13]:
# Generate a DataFrame with perturbed values for all rows in fft_test
num_rows_test = len(dataframe_testdata)
perturbed_test_df = add_perturbation(reference_features_test, num_rows_test, perturb_range=0.03)
perturbed_test_df

,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,spectral_flatness,phase_variance,phase_mean,phase_diff
0,539.358656,626.372752,48987.870819,4.270776,464.848697,6.379858e+06,9.193606,-59.650475,576666.861163,0.824143,3.248152,2.905423e-17,0.000262
1,532.786519,619.673325,49413.119761,4.217408,480.042531,6.320968e+06,9.002457,-58.805307,553077.683490,0.787831,3.310219,3.037064e-17,0.000253
2,546.292826,623.887105,47227.489363,4.234063,478.552838,6.295159e+06,8.955020,-58.053205,577448.317120,0.821915,3.238056,2.879710e-17,0.000260
3,526.735796,628.917299,46873.541911,4.299448,470.414871,6.408527e+06,9.174315,-60.312521,552074.691161,0.794065,3.272683,2.928302e-17,0.000260
4,527.924824,610.479937,48941.731575,4.298509,467.787339,6.347045e+06,9.353737,-58.948925,578138.942863,0.794675,3.266697,3.012753e-17,0.000254
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,547.271933,636.131051,49257.458275,4.213613,480.941931,6.490520e+06,9.193900,-59.090454,577105.908518,0.798895,3.172531,3.013866e-17,0.000260
11996,555.315646,622.749011,48526.658498,4.258186,483.643439,6.665124e+06,9.248427,-57.833091,556751.710165,0.833871,3.276848,3.037278e-17,0.000253
11997,536.372376,641.926929,48636.339729,4.297560,489.945984,6.421401e+06,9.085304,-59.964453,561263.518635,0.817903,3.325391,2.916552e-17,0.000265
11998,532.887503,610.666716,49508.194061,4.263168,492.350863,6.592954e+06,9.106078,-60.372945,550902.494754,0.800885,3.204730,2.929829e-17,0.000257


In [14]:
dataframe_testdata

,Frequency,FFT Magnitude,Phase,Infant_Presence
0,0.000000,48274.236998,0.000000,1.0
1,162.760417,24941.596497,-3.088069,1.0
2,325.520833,1716.246635,0.987059,1.0
3,488.281250,759.834659,-1.668255,1.0
4,651.041667,1026.975587,2.278972,1.0
...,...,...,...,...
11995,-813.802083,1362.883093,0.664906,1.0
11996,-651.041667,1026.975587,-2.278972,1.0
11997,-488.281250,759.834659,1.668255,1.0
11998,-325.520833,1716.246635,-0.987059,1.0


In [15]:
len(perturbed_test_df),len(dataframe_testdata)

(12000, 12000)

In [16]:
# Ensure both DataFrames have the same number of rows
if len(dataframe_testdata) != len(perturbed_test_df):
    print("The DataFrames have different lengths. Adjust them to have the same number of rows before concatenating.")
else:
    # Concatenate the DataFrames side by side
    dataframe_testdata_withfeatures = pd.concat([dataframe_testdata, perturbed_test_df], axis=1)

# Display the first few rows of the updated dataframe_testdata to verify
dataframe_testdata_withfeatures.head()

,Frequency,FFT Magnitude,Phase,Infant_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,spectral_flatness,phase_variance,phase_mean,phase_diff
0,0.000000,48274.236998,0.000000,1.0,539.358656,626.372752,48987.870819,4.270776,464.848697,6.379858e+06,9.193606,-59.650475,576666.861163,0.824143,3.248152,2.905423e-17,0.000262
1,162.760417,24941.596497,-3.088069,1.0,532.786519,619.673325,49413.119761,4.217408,480.042531,6.320968e+06,9.002457,-58.805307,553077.683490,0.787831,3.310219,3.037064e-17,0.000253
2,325.520833,1716.246635,0.987059,1.0,546.292826,623.887105,47227.489363,4.234063,478.552838,6.295159e+06,8.955020,-58.053205,577448.317120,0.821915,3.238056,2.879710e-17,0.000260
3,488.281250,759.834659,-1.668255,1.0,526.735796,628.917299,46873.541911,4.299448,470.414871,6.408527e+06,9.174315,-60.312521,552074.691161,0.794065,3.272683,2.928302e-17,0.000260
4,651.041667,1026.975587,2.278972,1.0,527.924824,610.479937,48941.731575,4.298509,467.787339,6.347045e+06,9.353737,-58.948925,578138.942863,0.794675,3.266697,3.012753e-17,0.000254


In [17]:
len(dataframe_testdata)

12000

# Testing the Random Forest Model

In [18]:
# dropping features

In [19]:
to_drop = {'median_fft', 'phase_mean', 'spectral_bandwidth', 'min_fft', 'max_fft', 'sum_fft', 'spectral_centroid', 'spectral_entropy', 'mean_fft', 'std_fft', 'phase_diff'}
test_X = dataframe_testdata_withfeatures.drop(columns=['Infant_Presence'])  # Features
test_y = dataframe_testdata_withfeatures['Infant_Presence']  # Labels
test_x_dataset_1_reduced = test_X.drop(columns=to_drop)

In [20]:
test_x_dataset_1_reduced

,Frequency,FFT Magnitude,Phase,spectral_flatness,phase_variance
0,0.000000,48274.236998,0.000000,0.824143,3.248152
1,162.760417,24941.596497,-3.088069,0.787831,3.310219
2,325.520833,1716.246635,0.987059,0.821915,3.238056
3,488.281250,759.834659,-1.668255,0.794065,3.272683
4,651.041667,1026.975587,2.278972,0.794675,3.266697
...,...,...,...,...,...
11995,-813.802083,1362.883093,0.664906,0.798895,3.172531
11996,-651.041667,1026.975587,-2.278972,0.833871,3.276848
11997,-488.281250,759.834659,1.668255,0.817903,3.325391
11998,-325.520833,1716.246635,-0.987059,0.800885,3.204730


In [21]:
test_y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
11995    1.0
11996    1.0
11997    1.0
11998    1.0
11999    1.0
Name: Infant_Presence, Length: 12000, dtype: float64

In [22]:
import os 
import joblib

# Load the trained Random Forest model
# clf = joblib.load('random_forest_model_Baby_Covered_in_Blanket_Sunscreen.pkl')

# Define path (adjust as needed)
model_dir = os.path.join("..", "..", "Models", "Baby Detection and Seat Classification Random Forest")
model_path = os.path.join(model_dir, "random_forest_model_Baby_Covered_in_Blanket_Sunscreen.pkl")

# Load the trained Random Forest model
clf = joblib.load(model_path)

# Example: Transform new data and make predictions
y_pred = clf.predict(test_x_dataset_1_reduced)
y_pred

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- spectral_bandwidth


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Optional: More detailed report
print("\nClassification Report:")
print(classification_report(test_y, y_pred))

# Optional: Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_y, y_pred))


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(test_y, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix for test dataset')
plt.show()

# Testing with XG Boost model task 3

In [ ]:
import os
import xgboost as xgb
import joblib


# Define paths
model_dir = os.path.join("..", "..", "Misc")
scalar_dir = os.path.join("..", "..", "Models", "Baby Detection and Seat Classification XGBoost")


# Model and scaler file paths
model_path = os.path.join(model_dir, "xgboost_model_Baby_Covered_in_Blanket_Sunscreen.json")
scaler_path = os.path.join(scalar_dir, "xgboost_scaler_Baby_Covered_in_Blanket_Sunscreen.pkl")

# Load the trained XGBoost model
clf = xgb.XGBClassifier()
clf.load_model(model_path)


# Load the scaler
scaler = joblib.load(scaler_path)

print(f"Model loaded from: {os.path.abspath(model_path)}")
print(f"Scaler loaded from: {os.path.abspath(scaler_path)}")

# Transform test data
X_new_scaled = scaler.transform(test_x_dataset_1_reduced)

# Predict
y_pred = clf.predict(X_new_scaled)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Optional: More detailed report
print("\nClassification Report:")
print(classification_report(test_y, y_pred))

# Optional: Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_y, y_pred))


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(test_y, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix for test dataset')
plt.show()